In [1]:
import re

# Load the book's text
with open('data/TKMBFullBook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
chapters = re.split(r'Chapter \d+', text)
if chapters[0].strip() == "":
    chapters.pop(0) # Remove the empty string before the first chapter

In [3]:
WINDOW = 384  # Number of words in each chunk #200
STRIDE = 60 # Number of words to shift for the next chunk # 40

def chapter_to_chunks(chapter_text):
    # Simple word splitting (preserving original text for transformer tokenizer)
    words = chapter_text.split()
    chunks = []
    
    for start in range(0, len(words), STRIDE):
        end = start + WINDOW
        if end > len(words):
            # For the last chunk, take all remaining words
            chunk_words = words[start:]
        else:
            chunk_words = words[start:end]
        
        # Join the words back into text with single spaces
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
    
    return chunks

all_chunks = []
for chap_num, chap_text in enumerate(chapters, start=1):
    for i, chunk in enumerate(chapter_to_chunks(chap_text), start=1):
        all_chunks.append({
            "chapter": chap_num,
            "chunk_id": f"{chap_num}_{i}",
            "text": chunk
        })
        
print(f"Total chunks created: {len(all_chunks)}")
print(f"Example chunk: {all_chunks[50]}")

Total chunks created: 1673
Example chunk: {'chapter': 1, 'chunk_id': '1_51', 'text': 'the Radleys: when Jem would question him Atticus’s only answer was for him to mind his own business and let the Radleys mind theirs, they had a right to; but when it happened Jem said Atticus shook his head and said, “Mm, mm, mm.” So Jem received most of his information from Miss Stephanie Crawford, a neighborhood scold, who said she knew the whole thing. According to Miss Stephanie, Boo was sitting in the livingroom cutting some items from The Maycomb Tribune to paste in his scrapbook. His father entered the room. As Mr. Radley passed by, Boo drove the scissors into his parent’s leg, pulled them out, wiped them on his pants, and resumed his activities. Mrs. Radley ran screaming into the street that Arthur was killing them all, but when the sheriff arrived he found Boo still sitting in the livingroom, cutting up the Tribune. He was thirty-three years old then. Miss Stephanie said old Mr. Radley said n

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

W0812 01:35:14.177000 48904 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [5]:
texts = [chunk['text'] for chunk in all_chunks]
embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True, batch_size=32)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

In [6]:
embeddings[1]  # Example to show the embedding of the second chunk

array([-1.10698612e-02,  2.98842490e-02,  2.10520942e-02, -2.23328304e-02,
       -1.47477062e-02,  2.19944287e-02, -5.79456706e-03, -2.03735996e-02,
       -3.41824368e-02, -3.51069979e-02,  3.03402339e-04,  6.35926723e-02,
        5.87773584e-02, -4.38749380e-02, -4.03507203e-02,  6.90658838e-02,
       -2.00750642e-02,  3.67354229e-02,  4.01172936e-02,  2.88017523e-02,
       -2.92388611e-02,  2.72633303e-02,  1.68018900e-02, -4.06044833e-02,
        1.24957960e-03,  2.80717164e-02,  3.14317527e-03,  3.81991081e-02,
       -1.33653833e-02, -3.70142870e-02, -2.30525732e-02, -1.21566392e-02,
       -5.28830141e-02, -5.90099841e-02,  2.55009900e-06,  1.47656482e-02,
        1.34951491e-02, -3.01998891e-02, -3.48994806e-02,  1.83912721e-02,
        8.74403343e-02,  6.55653775e-02,  7.53659522e-03, -6.02023909e-03,
       -5.96989468e-02, -1.26684252e-02, -4.99622189e-02,  6.23056144e-02,
       -1.01680048e-02,  3.44442390e-02, -5.47501550e-05,  1.39420209e-02,
        7.90802017e-03,  

In [7]:
embeddings.shape

(1673, 768)

Indexing the data with FIASS

In [8]:
import faiss
import numpy as np

# Convert to float32 (FAISS requires it)
embedding_matrix = np.array(embeddings).astype('float32')

# Build index (L2 or cosine - we'll use cosine)
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity (after normalization)

# Normalize for cosine similarity
faiss.normalize_L2(embedding_matrix)

# Add vectors
index.add(embedding_matrix)


In [9]:
def search_faiss(query, top_k=5):
    # Encode and normalize query
    query_vec = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)

    # Search index
    scores, indices = index.search(query_vec, top_k)

    # Retrieve the corresponding chunks
    results = []
    for i, score in zip(indices[0], scores[0]):
        result = {
            "chunk_id": all_chunks[i]["chunk_id"],
            "chapter": all_chunks[i]["chapter"],
            "text": all_chunks[i]["text"],
            "score": float(score)
        }
        results.append(result)
    
    return results


LLM to Query

In [10]:
import openai
from openai import OpenAI
import os

# Set up OpenAI API key
#openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this environment variable

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

#generate text
def generate_text(prompt: str) -> str:
    """Generates text using the OpenAI API based on the provided prompt."""
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # A powerful and cost-effective choice
        messages=[{"role": "system", "content": "You are a creative short story writer who writes stories in a specified book style and answer questions based on it."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,  # Adjust temperature for creativity
    )
    
    return response.choices[0].message.content.strip()

In [11]:
generated_text = generate_text("In 200 words, write a short story in the style of 'To Kill a Mockingbird' about a summer in manchester, UK.")
print(generated_text)

That summer in Manchester was unlike any other. The rain, usually a steady companion, had taken a rare holiday, leaving the city bathed in an unbroken stretch of sunshine. Jem and I, not much older than Scout and Jem Finch, roamed the red-brick streets like explorers charting unknown lands. Our playground was the cobbled alleys of Ancoats, where the scent of fresh bread mingled with the distant hum of factory whistles.

Mrs. Thompson, the elderly neighbor, watched us with a knowing smile, her eyes reflecting stories of a Manchester long past. She told us about the cotton mills and the voices of workers who forged the city’s soul. We listened, wide-eyed, while the afternoon sun painted golden patterns on the walls.

One day, we found a stray dog near the canal, skinny and wary. Jem wanted to leave it, but I saw something familiar in its cautious eyes—a survivor, just like us. We named him Scout and brought him home, a small act of kindness in a world that often forgot the gentle things.

In [12]:
query = generated_text
top_results = search_faiss(query, top_k=10)

for res in top_results:
    print(f"Chapter {res['chapter']} | Chunk {res['chunk_id']} | Score: {res['score']:.3f}")
    print(res['text'])
    print("---")


Chapter 9 | Chunk 9_35 | Score: 0.705
was a two-storied white house with porches circling it upstairs and downstairs. In his old age, our ancestor Simon Finch had built it to please his nagging wife; but with the porches all resemblance to ordinary houses of its era ended. The internal arrangements of the Finch house were indicative of Simon’s guilelessness and the absolute trust with which he regarded his offspring. There were six bedrooms upstairs, four for the eight female children, one for Welcome Finch, the sole son, and one for visiting relatives. Simple enough; but the daughters’ rooms could be reached only by one staircase, Welcome’s room and the guestroom only by another. The Daughters’ Staircase was in the ground floor bedroom of their parents, so Simon always knew the hours of his daughters’ nocturnal comings and goings. There was a kitchen separate from the rest of the house, tacked onto it by a wooden catwalk; in the back yard was a rusty bell on a pole, used to summon fie

Cross Encoder Reranking with FAISS

In [13]:
from sentence_transformers import CrossEncoder
_reranker = None

def get_reranker():
    global _reranker
    if _reranker is None:
        _reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    return _reranker

In [ ]:
# FAISS -> Cross-Encoder re-ranking
# topN: how many candidates to pull from FAISS before re-ranking
# final_topk: how many to return after re-ranking

from collections import defaultdict

def search_faiss_rerank(query, topN=100, final_topk=10, group_by_chapter=True, per_chapter=3, batch_size=32):
    # 1) Retrieve candidates with FAISS
    import faiss
    q = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q)
    scores, idxs = index.search(q, topN)

    candidates = []
    for j, i in enumerate(idxs[0]):
        candidates.append({
            "chunk_id": all_chunks[i]["chunk_id"],
            "chapter": all_chunks[i]["chapter"],
            "text": all_chunks[i]["text"],
            "retrieval_score": float(scores[0][j]),
            "_idx": int(i)
        })

    if not candidates:
        return []
      # 2) Re-rank with cross-encoder
    ce = get_reranker()
    pairs = [(query, c["text"]) for c in candidates]
    ce_scores = ce.predict(pairs, batch_size=batch_size)
    for c, s in zip(candidates, ce_scores):
        c["ce_score"] = float(s)

    # 3) Sort by CE score and slice
    candidates.sort(key=lambda x: x["ce_score"], reverse=True)
    top = candidates[:final_topk]

    if not group_by_chapter:
        return top

    # 4) chapter-aware grouping
    buckets = defaultdict(list)
    
    for c in top:
        buckets[c["chapter"]].append(c)

    cards = []
    for chap, items in buckets.items():
        items.sort(key=lambda x: x["ce_score"], reverse=True)
        chapter_score = max(i["ce_score"] for i in items)
        cards.append({
            "chapter": chap,
            "chapter_score": chapter_score,
            "chunks": items[:per_chapter]
        })

    cards.sort(key=lambda x: x["chapter_score"], reverse=True)
    return cards


In [15]:
Rerank_results = search_faiss_rerank(generated_text, topN=100, final_topk=10, group_by_chapter=True, per_chapter=3, batch_size=32)

In [16]:
#search_faiss_rerank
#sort the results by retrieval score
Rerank_results.sort(key=lambda x: x['chapter_score'], reverse=False)

In [17]:
Rerank_results

[{'chapter': 5,
  'chapter_score': -7.330019950866699,
  'chunks': [{'chunk_id': '5_11',
    'chapter': 5,
    'text': 'they paused in their pursuits: we reaped the benefits of a talent Miss Maudie had hitherto kept hidden from us. She made the best cakes in the neighborhood. When she was admitted into our confidence, every time she baked she made a big cake and three little ones, and she would call across the street: “Jem Finch, Scout Finch, Charles Baker Harris, come here!” Our promptness was always rewarded. In summertime, twilights are long and peaceful. Often as not, Miss Maudie and I would sit silently on her porch, watching the sky go from yellow to pink as the sun went down, watching flights of martins sweep low over the neighborhood and disappear behind the schoolhouse rooftops. “Miss Maudie,” I said one evening, “do you think Boo Radley’s still alive?” “His name’s Arthur and he’s alive,” she said. She was rocking slowly in her big oak chair. “Do you smell my mimosa? It’s like

Evaluation

In [18]:
#precision@k
def precision_at_k(results, k):
    """Calculates precision at k for the given results."""
    if not results or k <= 0:
        return 0.0
    
    relevant_count = sum(1 for res in results[:k] if res['retrieval_score'] > 0.5)  # Assuming a threshold for relevance
    return relevant_count / k

In [24]:
# Flatten the chunks from the grouped results
def precision_at_k_no_rerank(results, k):
	"""Calculates precision at k for the given results (no rerank)."""
	if not results or k <= 0:
		return 0.0
	relevant_count = sum(1 for res in results[:k] if res['score'] > 0.5)  # Use 'score' key
	return relevant_count / k

pk_no_rerank = precision_at_k_no_rerank(top_results, k=10)
print(f"Precision at k=10: {pk_no_rerank:.2f}")

Precision at k=10: 1.00


In [27]:
#recall@k
def recall_at_k_no_rerank(results, k):
    """Calculates recall at k for the given results."""
    if not results or k <= 0:
        return 0.0
    
    relevant_count = sum(1 for res in results[:k] if res['score'] > 0.5)  # Assuming a threshold for relevance
    total_relevant = sum(1 for res in results if res['score'] > 0.5)
    
    return relevant_count / total_relevant if total_relevant > 0 else 0.0

recall_no_rerank = recall_at_k_no_rerank(top_results, k=10)
print(f"Recall at k=10: {recall_no_rerank:.2f}")

Recall at k=10: 1.00


1.0